In [57]:
url = 'https://www.nseindia.com/archives/equities/mto/MTO_17122018.DAT'

In [58]:
df = requests.get(url)

In [48]:
with open('test_file.dat', 'wb') as f:
    f.write(df.content)
    f.close()

In [53]:
data = pd.read_csv('test_file.dat', skiprows=4, delimiter=',', names=['Record Type','Sr No','Name of Security','Series',
                                                                      'Quantity Traded','Deliverable Quantity(gross across client level)',
                                                                     '% of Deliverable Quantity to Traded Quantity'], na_filter = False)


In [54]:
data.drop(columns=['Record Type','Sr No'], inplace=True)


In [55]:
data['key'] = data['Name of Security']+'_'+data['Series']

In [56]:
data.to_csv('abc.csv')

In [1]:
def security_delivery_dumper(file_name):
    '''Func to dump security wise delivery from nse to cassandra'''
    
    # web scrap and get the data    
    url = 'https://www.nseindia.com/archives/equities/mto/'+file_name
    r = requests.get(url)
    trade_date, settlement_date = None, None
    if r.status_code == 200:
        # successfull connetion
        with open('temp.dat', 'wb') as f:
            # write the contents to temp dat file 
            f.write(r.content)
            f.close()

            date_extractor = str(r.content.split('\n')[2:3])
            trade_date =  dparser.parse(date_extractor[:25], fuzzy=True).date()
            settlement_date = dparser.parse(date_extractor[30:], fuzzy=True).date()
        
    else:
        logging.info('Dat not found for {0} day /{1} connection received from server...'.format(date,r.status_code))
        print 'Dat not found for {0} day /{1} connection received from server...'.format(date,r.status_code)
        return -1
    
    # read and process data to temp csv
    data = pd.read_csv('temp.dat', skiprows=4, delimiter=',', 
                       names=['Record Type','Sr No','Name of Security','Series',
                              'Quantity Traded','Deliverable Quantity(gross across client level)',
                              '% of Deliverable Quantity to Traded Quantity'], na_filter = False)
    
    data.drop(columns=['Record Type','Sr No'], inplace=True)
    # generate key to strore in cassandra db
    data['key'] = data['Name of Security']+'_'+data['Series']
    #get trade and settlement date
    data['Trade Date'] = trade_date
    data['Settlement Date'] = settlement_date
    data = data[['key','Trade Date','Settlement Date','Name of Security','Series',
                'Quantity Traded','Deliverable Quantity(gross across client level)',
                 '% of Deliverable Quantity to Traded Quantity']]
    data.to_csv('temp.csv', index=False)
    # processed file 
    data.to_csv(processed_dir+file_name)
    
    # write csv file to cassandra db
    os.system(root_dir+"dump.bat ")   
    return 1



def getFilename(d):
    '''Func to get filenames for given date'''
    return 'MTO_{0}{1}{2}.DAT'.format(date.strftime("%d"), date.strftime("%m"), date.strftime("%Y"))

In [2]:
import os
from cassandra.cluster import Cluster
import logging 
import pandas as pd
import datetime
import dateutil.parser as dparser
from time import time 
import requests
from dateutil import parser

root_dir = "X:\\Data_Analytics\\Security_wise_delivery\\"
processed_dir = "X:\\Data_Analytics\\Security_wise_delivery\\Processed_files\\"
log_path = "X:\\Data_Analytics\\Security_wise_delivery\\" 


# log events in debug mode 
logging.basicConfig(filename=log_path+"test.log",
                        level=logging.DEBUG,
                        format="%(asctime)s:%(levelname)s:%(message)s")



def security_delivery_dumper(file_name, date):
    '''Func to dump security wise delivery from nse to cassandra'''
    
    # web scrap and get the data    
    url = 'https://www.nseindia.com/archives/equities/mto/'+file_name
    r = requests.get(url)
    trade_date, settlement_date = None, None
    if r.status_code == 200:
        # successfull connetion
        logging.info('Successful connection : status_code 200')
        with open('temp.dat', 'wb') as f:
            # write the contents to temp dat file 
            f.write(r.content)
            f.close()

            date_extractor = str(r.content.split('\n')[2:3])
            trade_date =  dparser.parse(date_extractor[:25], fuzzy=True).date()
            logging.info('Traded date: {0}'.format(trade_date))
            settlement_date = dparser.parse(date_extractor[30:], fuzzy=True).date()
            logging.info('Settlement date: {0}'.format(settlement_date))
            print 'Successfully downloaded {0}...'.format(file_name)
        
    else:
        logging.info('Dat file not found for {0} day /{1} connection received from server...'.format(date,r.status_code))
        print 'Dat not found for {0} day /{1} connection received from server...'.format(date,r.status_code)
        return -1
    
    # read and process data to temp csv
    data = pd.read_csv('temp.dat', skiprows=4, delimiter=',', 
                       names=['Record Type','Sr No','Name of Security','Series',
                              'Quantity Traded','Deliverable Quantity(gross across client level)',
                              '% of Deliverable Quantity to Traded Quantity'], na_filter = False)
    
    data.drop(columns=['Record Type','Sr No'], inplace=True)
    # generate key to strore in cassandra db
    data['key'] = data['Name of Security']+'_'+data['Series']
    #get trade and settlement date
    data['Trade Date'] = trade_date
    data['Settlement Date'] = settlement_date
    data = data[['key','Trade Date','Settlement Date','Name of Security','Series',
                'Quantity Traded','Deliverable Quantity(gross across client level)',
                 '% of Deliverable Quantity to Traded Quantity']]
    data.to_csv('temp.csv', index=False)
    
    # processed file 
    data.to_csv(processed_dir+file_name)
    
    # write csv file to cassandra db
    os.system(root_dir+"dump.bat ")   
    logging.info('File successfully dumped to cassandra and moved to processed folder')
    return 1


def getFilename(date):
    '''Func to get filenames for given date'''
    return 'MTO_{0}{1}{2}.DAT'.format(date.strftime("%d"), date.strftime("%m"), date.strftime("%Y"))


def main():
    
    # create python cluster object to connect to your cassandra cluster (specify ip address of nodes to connect within your cluster)
    cluster = Cluster(['127.0.0.1'])
    logging.info('Cassandra Cluster connected...')
    # connect to your keyspace and create a session using which u can execute cql commands 
    session = cluster.connect('test_df')
    logging.info('Using test_df keyspace')
    
    # CREATE A TABLE; to dump MTO files 
    session.execute('CREATE TABLE IF NOT EXISTS security_wise_delivery (key VARCHAR, trade_date DATE, settlement_date DATE, Name_of_security VARCHAR, series VARCHAR, Quantity_Traded Decimal,Deliverable_Quantity_gross_client_level DECIMAL,Deliverable_Quantity_to_Traded_Quantity_percent DECIMAL, PRIMARY KEY (key, trade_date))')
    
    # check in processed folder and if not then dowload MTO file and dump in cassandra    
    errContinous = 0        
    d = datetime.date.today()
    decr = datetime.timedelta(days=1)
    while errContinous > -30 and (not os.path.exists(os.path.join(processed_dir,getFilename(d)))):
        if d == parser.parse('2016 01 01').date():
            break
        if security_delivery_dumper(getFilename(d), d) > -1:
            errContinous = 0
        else:
            errContinous -= 1
        
        d -= decr
        
             
        
        
start_time = time()
if __name__ == '__main__':
    main()

end_time = time()

logging.info('Time taken to process :'.format(end_time - start_time))
print "Execution time: {0} Seconds.... ".format(end_time - start_time)


Dat not found for 2018-12-26 day /404 connection received from server...
Dat not found for 2018-12-25 day /404 connection received from server...
Dat not found for 2018-12-23 day /404 connection received from server...
Dat not found for 2018-12-22 day /404 connection received from server...
Dat not found for 2018-12-16 day /404 connection received from server...
Dat not found for 2018-12-15 day /404 connection received from server...


KeyboardInterrupt: 

In [3]:
import datetime

In [15]:
datetime.date.today().strftime("%m")

'12'